# Google Calendar Toolkit

## Overview

> [Google Calendar](https://workspace.google.com/intl/en-419/products/calendar/) is a product of Google that allows users to organize their schedules and events. It is a cloud-based calendar that allows users to create, edit, and delete events. It also allows users to share their calendars with others.

This notebook will help you getting started with the Google Calendar Toolkit. This toolkit interacts with the Google Calendar API to perform various operations on the calendar. It allows you to create, search, update, and delete events on your calendar. It also allows you to list events on your calendar.

To use this toolkit, you will need to set up your credentials explained in the [Google Calendar API docs](https://developers.google.com/calendar/api/quickstart/python#authorize_credentials_for_a_desktop_application). Once you've downloaded the `credentials.json` file, you can start using the Google Calendar API.

## Setup

This toolkit requires the Google Python libraries, also, throughout the notebook, the following dependencies are used

In [ ]:
%pip install --upgrade --quiet google-api-python-client google-auth-httplib2 google-auth-oauthlib
%pip install --upgrade --quiet langchain
%pip install --upgrade --quiet langchain-community
%pip install --upgrade --quiet langchain-openai
%pip install --upgrade --quiet langgraph

## Instantiation

By default the toolkit reads the local `credentials.json` file. You can also manually provide a `Credentials` object.

In [1]:
from langchain_community.agent_toolkits import GoogleCalendarToolkit

toolkit = GoogleCalendarToolkit()

### Customizing Authentication

Behind the scenes, a `googleapi` resource is created using the following methods. you can manually build a `googleapi` resource for more auth control.

In [5]:
from langchain_community.agent_toolkits import GoogleCalendarToolkit
from langchain_community.tools.google_calendar.utils import (
    build_resource_service,
    get_google_calendar_credentials,
)

# Can review scopes here: https://developers.google.com/calendar/api/auth
credentials = get_google_calendar_credentials(
    token_file="token.json",
    scopes=["https://www.googleapis.com/auth/calendar"],
    client_secrets_file="credentials.json",
)
api_resource = build_resource_service(credentials=credentials)
toolkit = GoogleCalendarToolkit(api_resource=api_resource)

## Invocation

View available tools:

In [2]:
tools = toolkit.get_tools()
tools

[CalendarCreateEvent(api_resource=<googleapiclient.discovery.Resource object at 0x120f77350>),
 CalendarSearchEvents(api_resource=<googleapiclient.discovery.Resource object at 0x120f77350>),
 CalendarUpdateEvent(api_resource=<googleapiclient.discovery.Resource object at 0x120f77350>),
 GetCalendarsInfo(api_resource=<googleapiclient.discovery.Resource object at 0x120f77350>),
 CalendarMoveEvent(api_resource=<googleapiclient.discovery.Resource object at 0x120f77350>),
 CalendarDeleteEvent(api_resource=<googleapiclient.discovery.Resource object at 0x120f77350>)]

## Chaining

In [6]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)
response = llm_with_tools.invoke(
    [HumanMessage("Create a event tomorrow at 10am for meeting with John.")]
)

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'create_calendar_event', 'args': {'summary': 'Meeting with John', 'start_datetime': '2024-11-29 10:00:00', 'end_datetime': '2024-11-29 11:00:00'}, 'id': 'call_IJUCAmUhLPXwbCQGNZ4wlM1S', 'type': 'tool_call'}]


## Use within an agent

Below we show how to incorporate the toolkit into an Agent with `langgraph`

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [4]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [6]:
example_query = "Create an event with invite to fake@fake.com to \
invite him to have a coffee this afternoon"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Create an event with invite to fake@fake.com to invite him to have a coffee this afternoon
================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_Xs3ik5vOlYA53crrXbZslsGh)
 Call ID: call_Xs3ik5vOlYA53crrXbZslsGh
  Args:
    summary: Coffee Invitation
    start_datetime: 2024-11-27 15:00:00
    end_datetime: 2024-11-27 16:00:00
    attendees: ['fake@fake.com']
    description: Let's meet for coffee this afternoon!
================================= Tool Message =================================
Name: create_calendar_event

Event created: https://www.google.com/calendar/event?eid=bXZuM2syZ2ZiMjBsZXNlbzRraHE0YTczM28gam9yZ2VhbmczM0Bt
================================== Ai Message ==================================

I have created the event for a coffee invitation this afternoon and sent an invite to **fake@fake.com**. You can view the event [he

## API reference

Refer to the [Google Calendar API overview  |  Google for Developers](https://developers.google.com/calendar/api/guides/overview) for more details.